In [1]:
#make-training-data.ipynb
#
#by Joe Hahn
#joe.hahn@oracle.com
#3 July 2018
#
#generate training data

In [2]:
N_dots = 50#15000
x_half_width = 0.5
radius = 3.5
box_half_width = 7.1
jitter = 0.0
rn_seed = 13
initial_id = 123

In [3]:
import numpy as np
#import pandas as pd
rn_state = np.random.RandomState(seed=rn_seed)
from helper_fns import *

In [4]:
xo = make_xo_dict(x_half_width, radius, box_half_width, jitter)
xo

{'angle': 0.1581463935566021,
 'class': 'B',
 'id': -1,
 'r': 6.765791040556779,
 'x': 6.681360060799862,
 'y': 1.0655309204461672}

In [5]:
df = make_xo_df(N_dots, initial_id, x_half_width, radius, box_half_width, jitter)

NameError: name 'initial_id' is not defined

In [11]:
for idx in np.arange(5):
    print idx

0
1
2
3
4
